<a href="https://colab.research.google.com/github/KY-2000/StaffTracking_YOLOv8/blob/main/Footfallcam_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install ultralytics
!pip install shutil
!pip install cv2
!pip install numpy

from ultralytics import YOLO
from datetime import datetime
import shutil
import os
import cv2
# from utils.plots import plot_results

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# check if notebook connected to gpu
import tensorflow as tf
tf.test.gpu_device_name()

In [6]:
# train mode
# datetime for train filename
current_datetime = datetime.now().strftime("%Y-%m-%d %H-%M")
str_current_datetime = str(current_datetime)

# Load default YOLO model.
model = YOLO('yolov8s.pt') # pretrained model, can change to nano, large, huge etc.
model_epoch = 50 # change training epoch
model_batch = 4 # change training batch
model_name = 'yolov8s_staff' + "_epoch_" + str(model_epoch) + "_batch_" + str(model_batch) + "_" + str_current_datetime


log_file = "/content/drive/MyDrive/Colab Notebooks/log"
if not os.path.exists(log_file):
    os.mkdir(log_file)

project_dir = os.path.join(log_file, model_name)
dataset_dir = "/content/drive/MyDrive/Colab Notebooks/Staff_Dataset/staff.yaml"

# Train Custom Model
results = model.train(data=dataset_dir,
                      epochs=model_epoch, batch=model_batch,
                      project=project_dir)

"""
# save log and model
colab_link = os.path.join('/content/runs/detect/', model_name)
gdrive_link = os.path.join('/content/drive/MyDrive/Colab Notebooks/log', model_name)

def save_file_to_drive(filename, gdrive_link):
  for x in os.listdir(filename):
    dir_name = (os.path.join(filename, x))
    if os.path.exists(dir_name) and os.path.isdir(dir_name):
      save_file_to_drive(dir_name, os.path.join(gdrive_link, os.path.basename(os.path.normpath(dir_name))))
      continue
    if not os.path.exists(gdrive_link):
      os.mkdir(gdrive_link)
    shutil.copy(dir_name, gdrive_link)

save_file_to_drive(colab_link, gdrive_link)
"""

Ultralytics YOLOv8.0.173 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Colab Notebooks/Staff_Dataset/staff.yaml, epochs=50, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09-10 15-13, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=Fals

"\n# save log and model\ncolab_link = os.path.join('/content/runs/detect/', model_name)\ngdrive_link = os.path.join('/content/drive/MyDrive/Colab Notebooks/log', model_name)\n\ndef save_file_to_drive(filename, gdrive_link):\n  for x in os.listdir(filename):\n    dir_name = (os.path.join(filename, x))\n    if os.path.exists(dir_name) and os.path.isdir(dir_name):\n      save_file_to_drive(dir_name, os.path.join(gdrive_link, os.path.basename(os.path.normpath(dir_name))))\n      continue\n    if not os.path.exists(gdrive_link):\n      os.mkdir(gdrive_link)\n    shutil.copy(dir_name, gdrive_link)\n\nsave_file_to_drive(colab_link, gdrive_link)\n"

In [7]:
trained_model = project_dir + "/train/weights/best.pt"

# load trained model
model = YOLO(trained_model)

In [8]:
# Open the video file
video_path = "/content/drive/MyDrive/Colab Notebooks/sample.mp4" # change the directory of video file

In [9]:
# Track Mode
results = model.track(video_path, project=project_dir, name="video",save=True, save_txt=True, tracker="bytetrack.yaml")



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/1341) /content/drive/MyDrive/Colab Notebooks/sample.mp4: 480x640 (no detections), 118.2ms
video 1/1 (2/1341) /content/drive/MyDrive/Colab Notebooks/sample.mp4: 480x640 (no detections), 13.1ms
video 1/1 (3/1341) /content/drive/MyDrive/Colab Notebooks/sample.mp4: 480x640 (no detections), 13.0ms
video 1/1 (4/1341) /content/drive/MyDrive/Colab Notebooks/sample.mp4: 480x640 (no detections), 13.4ms
video 1/1 (5/1341) /content/drive/MyDrive/Colab

In [10]:
# Track Mode

cap = cv2.VideoCapture(video_path)
count = 0
frame_dir = os.path.join(project_dir, "frame")
if not os.path.exists(frame_dir):
  os.mkdir(frame_dir)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    count += 1

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, tracker="bytetrack.yaml")

        # Visualize the results on the frame
        frame = results[0].plot()
        if(results[0].boxes.xyxy.cpu().numpy().any()):
          # record frames which staff present
          txt_dir = os.path.join(frame_dir, "staff_frame.txt")
          f = open(txt_dir, "a+")
          f.write(str(count) + "\n")
          f.close()

          # draw midpoint of bbox in frame
          staff_coordinates = results[0].boxes.xyxy.cpu().numpy()
          staff_mid_x_list = []
          staff_mid_y_list = []
          for i in (range(len(staff_coordinates))):
            staff_mid_x = int((staff_coordinates[i][0] + staff_coordinates[i][2]) / 2)
            staff_mid_y = int((staff_coordinates[i][1] + staff_coordinates[i][3]) / 2)
            staff_mid_x_list.append(staff_mid_x)
            staff_mid_y_list.append(staff_mid_y)
          for i in range(len(staff_mid_x_list)):
            x = staff_mid_x_list[i]
            y = staff_mid_y_list[i]
            cv2.circle(frame, (x,y), radius=3, color=(0, 0, 255), thickness=-1)
            cv2.putText(frame, f'({x},{y})', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # save frame imgs
        frame_file = os.path.join(frame_dir, str(count) + ".jpg")
        cv2.imwrite(frame_file, frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object
cap.release()

Streaming output truncated to the last 5000 lines.

0: 480x640 1 staff, 16.3ms
Speed: 3.1ms preprocess, 16.3ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)
Results saved to /content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09-10 15-13/video
692 labels saved to /content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09-10 15-13/video/labels

0: 480x640 1 staff, 13.1ms
Speed: 2.7ms preprocess, 13.1ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
Results saved to /content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09-10 15-13/video
692 labels saved to /content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09-10 15-13/video/labels

0: 480x640 1 staff, 13.1ms
Speed: 3.6ms preprocess, 13.1ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)
Results saved to /content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09

In [11]:
# Export model
model = YOLO(trained_model)
model.export() # export to pytorch ver.

Ultralytics YOLOv8.0.173 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients

PyTorch: starting from '/content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09-10 15-13/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 4.0s, saved as '/content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09-10 15-13/train/weights/best.torchscript' (42.9 MB)

Export complete (6.4s)
Results saved to /content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09-10 15-13/train/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09-10 15-13/train/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=/content/drive/MyDr

'/content/drive/MyDrive/Colab Notebooks/log/yolov8s_staff_epoch_50_batch_4_2023-09-10 15-13/train/weights/best.torchscript'

In [ ]:
#Validation Mode

metrics = model.val(data=dataset_dir)  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps